In [1]:
pip install folium==0.12.1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os
import requests
import pprint
import json
import folium

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

서울지도

In [4]:
# params ---- 1.
center = [37.541, 126.986]
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']

# visualization ---- 2.
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


In [5]:
raw_data = [
            ['강동구',4], ['송파구',1], ['강남구',1], ['서초구',1], ['관악구',4],
            ['동작구',4], ['영등포구',4], ['금천구',3], ['구로구',4], ['강서구',4],
            ['양천구',4], ['마포구',4], ['서대문구',2], ['은평구',4], ['노원구',2],
            ['도봉구',4], ['강북구',3], ['성북구',3], ['중랑구',3], ['동대문구',4],
            ['광진구',1], ['성동구',4], ['용산구',2], ['중구', 4], ['종로구',4]           
]
data = pd.DataFrame(raw_data, columns=['name', 'value'])
data.head()

,name,value
0,강동구,4
1,송파구,1
2,강남구,1
3,서초구,1
4,관악구,4


In [6]:
geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
geo_json_data = json.loads(requests.get(geo_json).text)


m = folium.Map(
    location = [37.566345, 126.977893], 
    zoom_start = 10.5,
)

# GeoJson 레이어 추가
folium.GeoJson(
    geo_json_data,
    style_function=lambda x: {
        'fillColor': '#00000000',
        'color': 'black',
        'weight': 1
    }
).add_to(m)

folium.Choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=data,
    columns=['name', 'value'],
    key_on='feature.properties.name',
    fill_color='GnBu',  # 기본 색상으로 변경
    fill_opacity=0.5,
    line_opacity=0.2,
).add_to(m)
m

장애인 현황 지도에 찍기

In [7]:
import pandas as pd

# 파일 경로
file_path = './장애인+현황(장애유형별).xlsx'

# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

# '계'에 해당하는 열 인덱스 찾기 (첫 번째 '2022')
col_index = df.columns.get_loc("2022")  # '계'에 해당하는 첫 번째 '2022' 열 인덱스

# 데이터 선택 (첫 3행을 제외하고 '소계'가 아닌 모든 행 선택)
filtered_df = df.iloc[3:, [1, col_index]]

# 새로운 데이터프레임 생성
new_df = filtered_df.rename(columns={'자치구별(2)': '자치구별', df.columns[col_index]: '계'})

no_totals_df = new_df[new_df['자치구별'] != '소계']

no_totals_df.head()


FileNotFoundError: [Errno 2] No such file or directory: './장애인+현황(장애유형별).xlsx'

In [ ]:
admin_division_file_path = './korea_administrative_division_latitude_longitude.xlsx'
admin_div_df = pd.read_excel(admin_division_file_path)

# 데이터 확인을 위한 상위 5행 출력
admin_div_df.head()

,docity,do,city,longitude,latitude
0,강원강릉시,강원,강릉시,128.878497,37.749136
1,강원고성군,강원,고성군,128.470164,38.377961
2,강원동해시,강원,동해시,129.116633,37.521931
3,강원삼척시,강원,삼척시,129.167489,37.447086
4,강원속초시,강원,속초시,128.594167,38.204275


In [ ]:
# 서울시 자치구만 필터링
seoul_admin_div_df = admin_div_df[admin_div_df['do'] == '서울']

# 'city' 열을 기준으로 no_totals_df와 결합
# no_totals_df의 '자치구별' 열과 seoul_admin_div_df의 'city' 열이 일치해야 합니다.
# '자치구별' 열의 이름을 'city'로 변경하여 일치시킴
no_totals_df_renamed = no_totals_df.rename(columns={'자치구별': 'city'})

# Merge (결합) 작업
merged_df = pd.merge(seoul_admin_div_df, no_totals_df_renamed, on='city')

merged_df.head()

,docity,do,city,longitude,latitude,계
0,서울강남구,서울,강남구,127.049556,37.514575,15158
1,서울강동구,서울,강동구,127.125864,37.527367,18785
2,서울강북구,서울,강북구,127.027719,37.636956,17186
3,서울강서구,서울,강서구,126.851675,37.548156,28508
4,서울관악구,서울,관악구,126.953844,37.475386,20228


In [ ]:
import folium

# 서울시 중심의 기본 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 각 자치구별로 원형 마커 추가
for idx, row in merged_df.iterrows():
    
    radius = row['계'] / 1000 
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=radius, # 반지름
                        color='blue',
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.6,
                        popup=f"{row['city']}: {row['계']}").add_to(seoul_map)


In [ ]:
seoul_map

두 요소를 합쳐보기

In [ ]:
import folium
import pandas as pd

# 서울시 중심의 기본 지도 생성
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=10.5)

# Choropleth 레이어 추가
folium.Choropleth(
    geo_data=geo_json, 
    name='choropleth',
    data=data,  
    columns=['name', 'value'], 
    key_on='feature.properties.name',  
    fill_color='PuBu', 
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(seoul_map)

# CircleMarker 레이어 추가
for idx, row in merged_df.iterrows():
    radius = row['계'] / 1000
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color='black',
        fill=True,
        fill_color='white',
        fill_opacity=0.6,
        popup=f"{row['city']}: {row['계']}"
    ).add_to(seoul_map)

seoul_map
